In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## Help Functions

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

'''
Solve the Cubic Equations
'''
import math
import cmath
def ThreeSquare(x):
    if x.imag == 0:
        #real
        m = x.real
        ans = -math.pow(-m,1/3) if m<0 else math.pow(m,1/3)
    else:
        #complex
        ans = x**(1/3)
    return ans

# round to num
def RoundAns(x,num):
    if x.imag == 0:
        m = x.real
        ans = round(m,num)
    else:
        m = round(x.real,num)
        n = round(x.imag,num)
        ans = complex(m,n)
    return ans
    
def Cubic(args):
    a,b,c,d = args
    p = c/a-b**2/(3*a**2)
    q = d/a+2*b**3/(27*a**3)-b*c/(3*a**2)
    w = complex(-0.5,(3**0.5)/2)
    ww = complex(-0.5,-(3**0.5)/2)
    A = cmath.sqrt((q/2)**2+(p/3)**3)
    B = ThreeSquare(-q/2+A)
    C = ThreeSquare(-q/2-A)
    y1 = B+C
    y2 = w*B+ww*C
    y3 = ww*B+w*C
    D = b/(3*a)
    roots=[RoundAns(y1-D,6),RoundAns(y2-D,6),RoundAns(y3-D,6)]
    return roots

## For Case 1 and 2, without symmetric connectivity
* Based on determinant lemma, calculate eigenvalue outlier (spectrum of eigenvalues)
* Based on perturbation theorem, calculate the statistics of eigenvectors (eigenvector perturbations)
* Validate the results, in the case with homogeneous random gain and the case with heterogeneous random gain 

In [4]:
'''
Cases 1,2 without symmetric connectivity
Focus on standard deviation (variance)
Focus on second moment (self)
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,20,10,10
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' single eta '''
etaset=0*np.ones(3)
'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
meanReigAmE,meanReigAmI=np.mean(meigvecAm[:NE,0]),np.mean(meigvecAm[NE:,0])
meanLeigAmE,meanLeigAmI=np.mean(neigvecAm[:NE,0])/eigvAm[0],np.mean(neigvecAm[NE:,0])/eigvAm[0]
'''
for reference
'''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
'''
inverse S
'''
invB1=np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
X1,Y1=meigvecAm[:,1:].copy(),neigvecAm[:,1:].copy()
SAm = X1@invB1@Y1.T
'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
'''
>>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>>
'''
# celltype specific random gain
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
# xee,xei,xie,xii=1.0,0.5,0.3,0.1
'''
Recording Variables
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,ntrial,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2,N),dtype=complex),np.zeros((ngavg,ntrial,NE*2,N),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
Xprimebasexee=np.zeros((ntrial,NE*2,NE*2)) 
# Y -- identical to X when g=1.0 ([-1])
''' Iterative Processing '''
gaverageseries=np.linspace(0.1,1.0,ngavg)
for iktrial in range(ntrial):
    '''
    for each individual trial, supposed to continuously increase g from zero to one -- see the sensibility changing
    but within one single trial, the random matrix (also random structure) keeps unchanged, thus avoid changing in random pattern
    '''
    # >>> generate the (base/ref) random matrix for each trial
    Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xsym[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    XsymT=Xsym.copy().T
    Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xinit = Xsym.copy()
    # >>>>> Now, Xsym and XsymT keep the same while changing the random gain gaverage
    for idxeta,eta in enumerate(gaverageseries):
        #  >>> random gain continuously increases
        etaset=0.0*np.ones(3)#eta*np.ones(3)
        gaverage=eta        
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 
        '''## aamp = 0, XT --> 0 X --> 1'''
        aamp=np.sqrt(asqr) 
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()

        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # ''' heterogeneous '''
        X=Xinit.copy()
        X[:NE,:NE]*=(xee*gaverage/np.sqrt(2))
        X[:NE,NE:]*=(xei*gaverage/np.sqrt(2))
        X[NE:,:NE]*=(xie*gaverage/np.sqrt(2))
        X[NE:,NE:]*=(xii*gaverage/np.sqrt(2))

        '''
        >> start perturbed matrix
        >> * perturbed matrix: J; low rank structures: Am; random components: X
        >> * eigenvectors perturbed matrix: Reig, Leig --> renormalized by x0,y0,--> get xnorm0,ynorm0
        >> * low rank structure: x0,y0, 
        '''
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('eta full:',symtheofull,'; num:',symnumfull)
        '''
        statistics >>>>>>>>>>> Full connectivity
        '''
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        '''
        Sort Eigenvalue in ascending 
        '''
        eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
        idxsort=np.argsort(eigenvalue_Rsort)
        eigenvalue_sort = np.squeeze(eigvJ.copy())
        # eigvalseries[idxeta,iktrial,:] = eigenvalue_sort[idxsort]
        eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        meig = meig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        neig = neig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        Reig = meig.copy()
        Leig = neig.copy()
        normval=np.sum(Reig*Leig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=Leig.copy()/normval.copy()  # note that here we change from neig --> Leig, otherwise, if we use copy, there'll be some sort of mistakes

        '''
        normalization 0 (reference)
        '''
        tildex,tildey=np.reshape(Reig[:,0].copy(),(NE*2,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))
        # y --- left eigenvectors, are more likely to have different signs
        if (np.mean(tildey[:NE])*y0[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()

        '''  RECORDING STATES 
        * statesReig/Leig[grandom,trial,E/I/Total,mean/std]
        '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0]) 
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        ''' record the random derivative '''
        if idxeta == ngavg-1:
            print('recording ramdom matrix.')
            Xprimebasexee[iktrial,:,:]=X.copy()


normalization m & n
1.000000000000003 5.147815070493498
eigenvalues: [-2.5+0.j]
<ipython-input-4-34cfd14cce85>:68: ComplexWarning: Casting complex values to real discards the imaginary part
  invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
eta full: 0.0 ; num: 7.93203438984915e-06
eta full: 0.0 ; num: 4.337300726987772e-05
eta full: 0.0 ; num: 0.00010632291864008698
eta full: 0.0 ; num: 0.00019678176850047185
eta full: 0.0 ; num: 0.00031474955685101963
eta full: 0.0 ; num: 0.00046022628369174934
eta full: 0.0 ; num: 0.0006332119490226611
eta full: 0.0 ; num: 0.0008337065528437465
eta full: 0.0 ; num: 0.0010617100951550076
eta full: 0.0 ; num: 0.0013172225759564339
recording ramdom matrix.
eta full: 0.0 ; num: -2.8334559293828647e-05
eta full: 0.0 ; num: -4.17282018444464e-05
eta full: 0.0 ; num: -4.018092765188313e-05
eta full: 0.0 ; num: -2.369273671612485e-05
eta full: 0.0 ; num: 7.736370962836065e-06
eta full: 0.0 ; num: 5.410639538498817e-05
eta full: 0.0 ; num: 0.00011541733655033465
eta ful

* Recording the PARAMETERS, VARIABLES, DATA

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHomoZeroeta_data',x0=x0,y0=y0,Jparameters=Jparameters,Randgain=Randgain,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries,Xprimebasexee=Xprimebasexee)

* eigenvalue spectrum (ensemble averaging)

In [5]:
'''
without symmetry, 
eigvAm
'''
idxgavg,idxtrial=9,8
figtspt,axtspt=plt.subplots(figsize=(6,3))
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c='blue',alpha=0.125)  # >>>>>>>>>>>>>>>>>>>.
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.0,s=20,c='orange',marker='^',alpha=0.5)  # >>>>>>>>>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
#location of X 
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverageseries[idxgavg]
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
## effective gain only use ONE alpha(portion), average gain use alphas of sender and receiver
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
## >> zero symmetry >>
eta=0
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="darkorange", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>>>>>>>>>
avgg=np.sum(np.sum(gmat))*2/4.0
avgg=np.sqrt(avgg)
xa = avgg*np.cos(theta)
ya = avgg*np.sin(theta)
axtspt.plot(xa, ya, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse')
print(avgg,gaverage_)



1.0 1.0 1.0 1.0
0.9999999999999999
0.9999999999999999 0.9999999999999999
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


## individual perturbations of left and right eigenvectors (predictive accuracy)
* calculate the predicted perturbations (theo)
* comparing with numerical perturbations (num)

In [6]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''

'''
# loading data
# strfile='2021-03-13-15_20_04_perturbationHomoZeroeta_data'# homogeneous random gain
strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' # heterogeneous random gain
data = np.load(strfile+'.npz')
gaverageseries = data['gaverageseries']
x0,y0=data['x0'],data['y0']
meigvecAm,neigvecAm=data['meigvecAm'],data['neigvecAm']
# perturbed
Leigvecseries,Reigvecseries=data['Leigvecseries'],data['Reigvecseries']
Xprimebasexee=data['Xprimebasexee']
eigvAm=data['eigvAm']
ngavg,ntrial,N=np.shape(Leigvecseries)
Leigvecseries=np.expand_dims(Leigvecseries,axis=3)
Reigvecseries=np.expand_dims(Reigvecseries,axis=3)
NE=int(N/2.0)
'''

'''
calculate S0
'''
xticks = np.linspace(-0.08,0.08,2)
xlims = [-0.08,0.08]
yticks = np.linspace(-0.08,0.08,2)
ylims = [-0.08,0.08]
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvAm[i+1]
    Z[i,i]=1/(-eigvAm[0])
S0=np.zeros((NE*2,NE*2))
R1,L1=np.squeeze(meigvecAm[:,1:]),np.squeeze(neigvecAm[:,1:])
S0=R1@Z@L1.T

'''
# CHOOSE ONE TRIAL
'''
## EIGENVECTOR USED HERE :0 -- the last dimension, not reloading from .npy
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

<ipython-input-6-8e948e22722f>:36: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i,i]=1/(-eigvAm[0])
<ipython-input-6-8e948e22722f>:54: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(y0)
<ipython-input-6-8e948e22722f>:55: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(x0)
<ipython-input-6-8e948e22722f>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_the[:,i] = lprime_the*gsamples[i]
<ipython-input-6-8e948e22722f>:58: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_the[:,i] = rprime_the*gsamples[i]
<ipython-input-6-8e948e22722f>:54: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecse

## population statistics 
* first determint lemma for eigenvalue outlier -- no change
* perturbation theorem for variance of eigenvalue
* perturbation theorem for variance of left and right eigenvectors

In [7]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
## calculating the statistics of perturbation lambda
# Coding uses only TWO POINT(starting S and ending E) [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:,0],axis=1)
figE,axE=plt.subplots(figsize=(4,4))
cm='rbg'
axE.plot(gaverageseries,std_perturb_Lambda_num,color=cm[1],linewidth=1.5,label='numerical')
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='darkgreen',linewidth=1.5,label='analytical')
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.00,0.08]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.08,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)

figR0,axR0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axR0[0].scatter(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',label='numerical')
axR0[1].scatter(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',label='numerical')
axR0[0].scatter(gaverageseries,std_perturbReig_linear_numSE[:,0],color='orange',label='numerical')
axR0[1].scatter(gaverageseries,std_perturbReig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR0[0].set_title(r'$\sigma_{r}^E$')
axR0[1].set_title(r'$\sigma_{r}^I$')

figR,axR=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
for i in range(ntrial):
    axR[0].scatter(gaverageseries,statesReig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axR[1].scatter(gaverageseries,statesReig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axR[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR[0].set_title(r'$\sigma_{r}^E$')
axR[1].set_title(r'$\sigma_{r}^I$')

ylims=[-0.00,0.015]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.015,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR[i].set_xlim(xlims)
    axR[i].set_xticks([])
    axR[i].set_ylim(ylims)
    axR[i].set_yticks(yticks)

    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR[1].set_xticks(xticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(statesLeig[ngavg-1,:,0,1]),np.mean(statesLeig[ngavg-1,:,1,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)

figL0,axL0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axL0[0].scatter(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',label='numerical')
axL0[1].scatter(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',label='numerical')
axL0[0].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,0],color='orange',label='numerical')
axL0[1].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL0[0].set_title(r'$\sigma_{l}^E$')
axL0[1].set_title(r'$\sigma_{l}^I$')

figL,axL=plt.subplots(2,1,figsize=(4,4))
cm='mg'#'rb'#
# >>>>>>>> numerical solution
for i in range(ntrial):
    axL[0].scatter(gaverageseries,statesLeig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axL[1].scatter(gaverageseries,statesLeig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axL[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL[0].set_title(r'$\sigma_{l}^E$')
axL[1].set_title(r'$\sigma_{l}^I$')

ylims=[-0.00,0.025]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.025,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL[i].set_xlim(xlims)
    axL[i].set_xticks([])
    axL[i].set_ylim(ylims)
    axL[i].set_yticks(yticks)

    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL[1].set_xticks(xticks)
axL0[1].set_xticks(xticks)

std delta_lambda numerical: 0.049064794754072244
std of perturbation right eigenvector, num: [0.01151058 0.01132108] ; theo: [0.01032796 0.01032796]
std of perturbation left eigenvector, num: [0.02377017 0.02389549] ; theo: [0.02126656 0.02126656]
<ipython-input-7-e9e5c73b50a3>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
<ipython-input-7-e9e5c73b50a3>:58: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
<ipython-input-7-e9e5c73b50a3>:72: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)
D:\UseApp\An